In [164]:
import pandas as pd
# import, sort and prepare data:
price = pd.read_csv("prices.csv")
sale = pd.read_csv("sales.csv")
price["updated_at"] = pd.to_datetime(price["updated_at"])
sale['ordered_at'] = pd.to_datetime(sale['ordered_at'])
price = price.sort_values('updated_at', ascending = True)
sale = sale.sort_values('ordered_at', ascending = True)

## conduct to merge 2 tables: 
new_sale = pd.merge_asof(sale,price, left_on ='ordered_at', right_on='updated_at', by='product_id', direction ='nearest')
table_a = new_sale.loc[new_sale['ordered_at'] <= new_sale['updated_at'], ['product_id','old_price','quantity_ordered']]
table_b = new_sale.loc[new_sale['ordered_at'] > new_sale['updated_at'], ['product_id','new_price','quantity_ordered']]

### create final_date & calculate revenue:
table_a['new_price'] = table_a['old_price'] #rename column to concat
table_a = table_a[['product_id','new_price','quantity_ordered']] #exclude old_price column 

concat_data = pd.concat([table_a,table_b])
final_data = concat_data.groupby(['product_id','new_price'],as_index=False)['quantity_ordered'].count()
final_data['revenue'] = final_data['new_price'] * final_data['quantity_ordered']
final_data[['product_id','new_price','revenue']]

,product_id,new_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,214500
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2086500
9,4085861,58000,2146000


In [165]:
## total of revenue by product_id
final_data.groupby('product_id',as_index=False)['revenue'].sum()

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,264000
3,4085861,8136000
